In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from loader.TrainValLoader import train_val_loader
from loader.CSVLoader import csv_loader
from loader.TestLoader import test_loader
layers = 90
gt,wave = csv_loader()
all_data = train_val_loader()
all_data_test = test_loader()

In [3]:
import numpy as np
from prep.BM3DBlock import bm3d_block
from prep.Normlize import normalize_layers
from prep.WaveletBlock import wavelet_transform
from prep.ContinuumRemovalBlock import continuum_removal
from prep.GaussianBlock import gaussian_block
from prep.ResizeBlock import make_resize_block
from prep.HelperBlocks import pipe
from prep.PCABlock import pca_block
from prep.SpectralIndexes import calc_indexes
from prep.SmoothDerivativeBlock import savgol_block
from selMethods.FCBF import fcbf_method
from selMethods.LowestInterCor import lowest_inter_cor_method
from selMethods.MutualInfo import mutual_info_method
from selMethods.SpearmanCor import spearman_method
from selMethods.reliefF import reliefF_method

transform_set1 = lambda d, is_test: pipe(d,[
    make_resize_block(16, pre_fill=0, order=0),
    lambda d: savgol_block(d,5,2,1,2),
    lambda d: np.ma.filled(d,0)
]) 

transform_set2 = lambda d, is_test: pipe(d,[
    make_resize_block(24, pre_fill=0, order=0),
    lambda d: savgol_block(d,5,2,1,2),
    lambda d: np.ma.filled(d,0)
]) 

transform_set3 = lambda d, is_test: pipe(d,[
    make_resize_block(24, pre_fill=0, order=0,extend_strategy="padding"),
    lambda d: savgol_block(d,5,2,1,2),
    lambda d: np.ma.filled(d,0)
]) 

transform_set4 = lambda d, is_test: pipe(d,[
    make_resize_block(10, pre_fill=-1, order=0,extend_strategy="padding"),
    lambda d: savgol_block(d,5,2,1,2),
    lambda d: np.ma.filled(d,0)
]) 

transform_set5 = lambda d, is_test: pipe(d,[
    make_resize_block(10, pre_fill=None, order=0,extend_strategy="padding"),
    lambda d: savgol_block(d,5,2,1,2),
    lambda d: np.ma.filled(d,0)
]) 

transform_set6 = lambda d, is_test: pipe(d,[
    make_resize_block(10, pre_fill="harmonic", order=0, extend_strategy="padding",post_fill=0),
    lambda d: savgol_block(d,5,2,1,2),
    lambda d: np.ma.filled(d,0)
]) 

transform_set7 = lambda d, is_test: pipe(d,[
    make_resize_block(24, pre_fill=0, order=2, extend_strategy="padding"),
    lambda d: savgol_block(d,5,2,1,2),
    lambda d: np.ma.filled(d,0)
]) 

transform_set8 = lambda d, is_test: pipe(d,[
    make_resize_block(24, pre_fill=0, order=2, extend_strategy="padding"),
    lambda d: savgol_block(d,11,2,2),
    lambda d: normalize_layers(d,not is_test),
    lambda d: np.ma.filled(d,0)
]) 

In [4]:
transform_function = transform_set8
name="test8"

In [5]:

transformed, transform_next, indices = spearman_method(transform_function(all_data,False),gt["K"],final_count=30)
transformed_test = transform_next(transform_function(all_data_test,True))
np.save(f"./experiment4/{name}_K",transformed)
np.save(f"./experiment4/{name}_K_test",transformed_test)

In [6]:
transformed, transform_next, indices = reliefF_method(transform_function(all_data,False),gt["P"],final_count=30)
transformed_test = transform_next(transform_function(all_data_test,True))
np.save(f"./experiment4/{name}_P",transformed)
np.save(f"./experiment4/{name}_P_test",transformed_test)

In [7]:
transformed, transform_next, indices = fcbf_method(transform_function(all_data,False),gt["Mg"],final_count=60)
transformed_test = transform_next(transform_function(all_data_test,True))
np.save(f"./experiment4/{name}_Mg",transformed)
np.save(f"./experiment4/{name}_Mg_test",transformed_test)